In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

dataset, dataset_info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
dataset_info
test_set, training_set, validation_set = dataset['test'], dataset['train'], dataset['validation']

Found GPU at: /device:GPU:0


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/2.1.1.incomplete3I94GO/oxford_flowers102-test.tfrecord*.…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/2.1.1.incomplete3I94GO/oxford_flowers102-validation.tfre…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


Importing TensorFlow 

In [ ]:
num_classes = dataset_info.features['label'].num_classes
num_training_examples = 0
num_validation_examples = 0
for example in training_set:
    num_training_examples += 1
for example in validation_set:
    num_validation_examples += 1


In [ ]:
IMAGE_RES = 224
def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    return image, label
BATCH_SIZE = 32
train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)


In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 102:
      activation = "softmax"
      units = 102
    else:
      activation = "sigmoid"
      units = 1

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


In [ ]:
model = make_model(input_shape=(224,224) + (3,), num_classes=102)
#keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 50

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_batches,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_batches,
)

Epoch 1/50
32/32 [==============================] - 29s 641ms/step - loss: 0.5999 - accuracy: 0.8422 - val_loss: 130.6268 - val_accuracy: 0.0098
Epoch 2/50
32/32 [==============================] - 20s 606ms/step - loss: 0.3592 - accuracy: 0.9294 - val_loss: 150.5661 - val_accuracy: 0.0098
Epoch 3/50
32/32 [==============================] - 19s 585ms/step - loss: 0.2821 - accuracy: 0.9471 - val_loss: 224.0984 - val_accuracy: 0.0098
Epoch 4/50
32/32 [==============================] - 19s 580ms/step - loss: 0.2186 - accuracy: 0.9618 - val_loss: 156.0120 - val_accuracy: 0.0098
Epoch 5/50
32/32 [==============================] - 20s 617ms/step - loss: 0.1985 - accuracy: 0.9647 - val_loss: 169.9194 - val_accuracy: 0.0098
Epoch 6/50
32/32 [==============================] - 20s 628ms/step - loss: 0.1903 - accuracy: 0.9637 - val_loss: 168.1889 - val_accuracy: 0.0098
Epoch 7/50
32/32 [==============================] - 19s 590ms/step - loss: 0.1454 - accuracy: 0.9745 - val_loss: 147.0194 - val_ac

KeyboardInterrupt: ignored